In [ ]:
import pandas as pd
pd.set_option("display.precision", 2)
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

## df_indic_calculados

In [ ]:
df_indic_calculados = pd.read_csv('../entrada/todos_indicadores.csv',encoding="ISO-8859-1")
df_indic_calculados.head(2)

In [ ]:
df_indic_alto_nivel = pd.read_csv('../entrada/indicadores_alto.csv',encoding="ISO-8859-1")
df_indic_alto_nivel.head(2)

In [ ]:
df_idss_alto_nivel = pd.read_csv('../entrada/idss_alto_nivel.csv',encoding="ISO-8859-1")
df_idss_alto_nivel.columns = ['Registro_ANS', 'ANO', 'TRIMESTRE', 'porte', 'alto_nivel']
df_idss_alto_nivel.head(2)

In [ ]:
def get_df_filtrado_by_chave(df,ano,trimestre,porte,Verbose=False):
    if(Verbose == True):
        print("Filtrando por: ",ano,trimestre,porte)
    return df[(df['ANO'] ==  ano) \
                & (df['TRIMESTRE'] == trimestre) \
                & (df['porte'] == porte)]

In [ ]:
def eh_alto_nivel(x):
    df_alto_nivel = df_idss_alto_nivel[(df_idss_alto_nivel['ANO'] == x['ANO']) \
                                      & (df_idss_alto_nivel['TRIMESTRE'] == x['TRIMESTRE'])
                                      & (df_idss_alto_nivel['Registro_ANS'] == x['REG_ANS'])
                                      ]
    if len(df_alto_nivel) > 0:
        x['alto_nivel'] = 1
        x['n_alto_nivel'] = 0
    else:
        x['n_alto_nivel'] = 1
        x['alto_nivel'] = 0
    return x

In [ ]:
df_indic_calculados = df_indic_calculados.apply(lambda x: eh_alto_nivel(x),axis=1)

In [ ]:
df_indic_calculados.head()

## Gerando gráfico para verificar porcentagem de cooperativas avaliadas com a melhor faixa do IDEF/IDSM

In [ ]:
def gerar_grafico_alto_nivel_porte(df, nome_porte):
    N = len(df)
    n_alto_nivel = ((df['n_alto_nivel']/df['total'])*100).tolist()
    alto_nivel =  ((df['alto_nivel']/df['total'])*100).tolist()
    ind = np.arange(N)
    width = 0.35 
    p2 = plt.bar(ind,n_alto_nivel,width)
    p1 = plt.bar(ind, alto_nivel, width,bottom=n_alto_nivel)
    
    plt.ylabel('% de cooperativas')
    plt.title('Porcentagem de cooperativas de '+nome_porte+' nota de IDEF/IDSM')
    plt.xticks(ind, df.index)
    plt.legend((p1[0], p2[0]), ('>= 0,8', '< 0,8'),)
    plt.show()

In [ ]:
for porte in df_indic_calculados['porte'].unique():
    df_agrupado_por_porte = df_indic_calculados[(df_indic_calculados['porte'] == porte) & (df_indic_calculados['ANO'] < 2018)]
    df_agrupado_por_porte = df_agrupado_por_porte.groupby(['ANO'])['alto_nivel','n_alto_nivel'].sum()
    df_agrupado_por_porte['total'] = df_agrupado_por_porte['alto_nivel']+df_agrupado_por_porte['n_alto_nivel']
    gerar_grafico_alto_nivel_porte(df_agrupado_por_porte,porte)

#### Mostra a relação entre o resultado líquido e o total das receitas com operação de planos de saúde (contraprestações efetivas)

In [ ]:
def get_indicador_melhor_avaliacao(ano,trimestre,porte,indicador):
    df_result = df_indic_alto_nivel[(df_indic_alto_nivel["ANO"] == ano) &\
                                    (df_indic_alto_nivel["TRIMESTRE"] == trimestre) &\
                                    (df_indic_alto_nivel["porte"] == porte)]
    if (len(df_result) ==1):
        return df_result[indicador].sum()
    else:
        raise NameError('Resultado não único!!!')

In [ ]:
df_sumariza_indicadores = df_indic_alto_nivel.copy()

In [ ]:
colunas_indicadores = df_indic_alto_nivel.columns[3:]

In [ ]:
colunas_indicadores

In [ ]:
colunas_indicadores2 = ['{0}_melhor_aval'.format(i) for i in colunas_indicadores]

In [ ]:
df_sumariza_indicadores.columns = df_sumariza_indicadores.columns[:3].tolist() + colunas_indicadores2

In [ ]:
df_sumariza_indicadores = df_sumariza_indicadores.set_index(["ANO","TRIMESTRE",'porte'])

In [ ]:
def contar_cooperativas_acima_media(x):
    ano,trimestre,porte = x.name
    for indic in df_indic_alto_nivel.columns[3:]:
        melhor_avaliacao = get_indicador_melhor_avaliacao(ano,trimestre,porte,indic)
        df_by_chave = get_df_filtrado_by_chave(df_indic_calculados,ano=ano,trimestre=trimestre,porte=porte)
        x[indic+"_qtde_acima"] = len(df_by_chave[df_by_chave[indic] > melhor_avaliacao])        
        x[indic+"_qtde_abaixo"] = len(df_by_chave[df_by_chave[indic] <= melhor_avaliacao])
        x[indic+"_media_geral"] = df_by_chave[indic].mean()
    return x

In [ ]:
df_summarizado2 = df_sumariza_indicadores.apply(lambda x: contar_cooperativas_acima_media(x),axis=1)

In [ ]:
df_summarizado2.head()

In [ ]:
df_summarizado2 = df_summarizado2.reset_index()

In [ ]:
def gerar_periodo(x):
    ano =  str(x['ANO'])
    trimestre = str(x['TRIMESTRE']).zfill(2)
    return ano+trimestre
    

In [ ]:
df_summarizado2['PERIODO'] = df_summarizado2.apply(lambda x: gerar_periodo(x),axis=1)

In [ ]:
for _indic in colunas_indicadores:
    for porte in df_summarizado2['porte'].unique():
        print("############ Indicador ",_indic," para cooperativas de ",porte,"############ ")
        df_porte = df_summarizado2[(df_summarizado2['porte'] == porte) \
                                  & (df_summarizado2['ANO'] < 2018)]
        labels = df_porte["PERIODO"].tolist()
        ymelhor = df_porte[_indic+'_melhor_aval'].tolist()
        ygeral = df_porte[_indic+'_media_geral'].tolist()
        

        fig, (ax1, ax2) = plt.subplots(nrows=2,ncols=1)
        fig.set_size_inches(20, 12)

        #--grafico de linha--#
        p1, = ax1.plot(labels, ygeral,color='black',linewidth=2.0)
        p2, = ax1.plot(labels, ymelhor, color='green',linewidth=2.0)
        
        ax1.title.set_text('Comparação média do indicador '+_indic+' das cooperativas de '+porte+' melhor avaliadas com média geral por ano')
        ax1.title.set_fontsize(20)
        ax1.legend([p1,p2],['Média Geral','Média melhor avaliadas'])
        #--------------------------------------------------------------------------------#

        y_acima_media =  df_porte[_indic+"_qtde_acima"].astype(int).tolist()
        y_abaixo_media =  df_porte[_indic+"_qtde_abaixo"].astype(int).tolist()

        x = np.arange(len(labels))  
        width = 0.35  

        rects1 = ax2.bar(x - width/2, y_acima_media, width, label='Acima da média')
        rects2 = ax2.bar(x + width/2, y_abaixo_media, width, label='Abaixo da média')

        ax2.title.set_text("Quantidade de cooperativas em comparação a média das melhores avaliadas")
        ax2.title.set_fontsize(20)
        ax2.set_xticks(x)
        ax2.set_xticklabels(labels)
        ax2.legend()


        def autolabel(rects):
            """Attach a text label above each bar in *rects*, displaying its height."""
            for rect in rects:
                height = rect.get_height()
                ax2.annotate('{}'.format(height),
                            xy=(rect.get_x() + rect.get_width() / 2, height),
                            xytext=(0, 3),  # 3 points vertical offset
                            textcoords="offset points",
                            ha='center', va='bottom')


        autolabel(rects1)
        autolabel(rects2)
        
       
        plt.tight_layout()

        plt.show()
        print("--------------------------------------------------------------------------------------------------------------------------")